In [ ]:
import pandas as pd
import numpy as np
import ndjson
import os
import sys
import pickle
from glob import glob
from tqdm import tqdm
import trajnetplusplustools
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
%matplotlib inline

In [ ]:
class FeatUtil(object):
    @classmethod
    def expand_dim(cls, ndarr, dtype=np.float32):
        return np.insert(ndarr, 2, values=0, axis=-1).astype(dtype)
    
    @classmethod
    def inter_one_man(cls, pos):
        return pd.DataFrame(pos).interpolate(limit_direction='both').values[:,np.newaxis,:]
    
    @classmethod
    def calc_vel(cls, pos):
        return np.diff(pos, axis=0)[:,np.newaxis,:]
    
    @classmethod
    def expand_particle(cls, arr, max_num, axis, value_type='int'):
        dummy_shape = list(arr.shape)
        if max_num <= arr.shape[axis]:
            return arr[:max_num,...]
        else:
            dummy_shape[axis] = max_num - arr.shape[axis]
            dummy = np.zeros(dummy_shape)
            if value_type == 'str':
                dummy = np.array(['dummy' + str(i) for i in range(np.product(dummy_shape))]).reshape(dummy_shape)
            return np.concatenate([arr, dummy], axis=axis)
    
    def __call__(self, pos):
        feat = {}
        
        # pos = np.concatenate([self.inter_one_man(pos[:,i,:]) for i in range(pos.shape[1])], axis=1)
        
        # drop distant drop nan
        pos, _ = drop_distant(pos)
        pos = pos[:,~np.isnan(pos).any(0).any(-1)].transpose(1,0,2)
        
        # vel = np.concatenate([self.calc_vel(pos[:,i,:]) for i in range(pos.shape[1])], axis=1)
        vel = np.diff(pos, axis=1)
        pos = pos[:,1:]
        
        mask = np.zeros(160)
        mask[:pos.shape[0]] = 1
        feat['man_mask'] = mask
        
        pos = self.expand_particle(pos, 40, 0)
        #pos = self.expand_dim(pos)
        vel = self.expand_particle(vel, 40, 0)
        #vel = self.expand_dim(vel)
        
        # pos = pos.transpose(1,0,2)
        # vel = vel.transpose(1,0,2)
        
        feat['pos_enc'] = pos[:,:7]
        feat['vel_enc'] = vel[:,:7]
        
        for i in range(7, pos.shape[1]):
            feat['pos'+str(i-7)] = pos[:,i,:]
            feat['vel'+str(i-7)] = vel[:,i,:]
            
        return feat

def drop_distant(xy, r=6.0):
    """
    Drops pedestrians more than r meters away from primary ped
    """
    distance_2 = np.sum(np.square(xy - xy[:, 0:1]), axis=2)
    mask = np.nanmin(distance_2, axis=0) < r**2
    return xy[:, mask], mask

In [ ]:
feat_util = FeatUtil()

In [ ]:
trainsource = '../pedestrian/train/real_data/'

file_list = glob(trainsource+ '*') 
for f in tqdm(file_list):
    reader = trajnetplusplustools.Reader(f, scene_type='paths')
    file_name = os.path.split(f)[1].split('.')[0]
    
    for s_id, s in reader.scenes():
        pos0 = trajnetplusplustools.Reader.paths_to_xy(s)
        feat = feat_util(pos0)
        feat['scene_idx'] = file_name + '_{}'.format(s_id)
        with open('../pedestrian/processed/train/' + file_name + '_{}.pkl'.format(s_id), 'wb') as f:
            pickle.dump(feat, f)

In [ ]:
testsource = '../pedestrian/test/real_data/'

file_list = glob(testsource+ '*') 
for f in tqdm(file_list):
    reader = trajnetplusplustools.Reader(f, scene_type='paths')
    file_name = os.path.split(f)[1].split('.')[0]
    
    for s_id, s in reader.scenes():
        pos0 = trajnetplusplustools.Reader.paths_to_xy(s)
        feat = feat_util(pos0)
        break
        feat['scene_idx'] = file_name + '_{}'.format(s_id)
        with open('../pedestrian/processed/val/' + file_name + '_{}.pkl'.format(s_id), 'wb') as f:
            pickle.dump(feat, f)